In [2]:
import numpy as np
import pandas as pd
import random
import scipy as sc
from sklearn import svm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import scale
import matplotlib
import matplotlib.pyplot as plt

In [1]:
####preparing data
Budget_range=[ 5,20,40,60,80,100,120,140,160,180,200]

myrandommatrix=pd.read_csv('myrandommatrix.csv')
myrandommatrix=myrandommatrix.values
memory_strength=0.02  
subjects_all=[1,2,3,4,5,6,7,8]


activities=['sit','stand', 'lying_back', 'ascend_stairs','descend_stairs', 'run_treadmill', 'stepper', 'cycle_horizontal','rowing', 'jumping']

for subjects in subjects_all:
    acc_all=[]
    presic_all=[]
    recall_all=[]
    f1_all=[]
    B2=0
    myiteration=10

    for Budget in Budget_range:
        acc_result_sum=0
        presic_result_sum=0
        recall_result_sum=0
        f1_result_sum=0

        for myx in range(myiteration):
            myrandomlist=myrandommatrix[myx,:]

            if Budget==Budget_range[0]:
                train_whole=pd.read_csv('.\\sub%s-train.csv'%subjects)
                train_labeled_whole=pd.read_csv('.\\sub%s-train-labeled.csv'%subjects)
                train_unlabeled_whole=pd.read_csv('.\\sub%s-train-unlabeled.csv'%subjects)
                train_unlabeled_whole=train_unlabeled_whole.dropna()
                index_col=train_labeled_whole.shape[1]

                test_whole=pd.read_csv('.\\sub%s-test.csv'%subjects)
                test=test_whole.iloc[:, 0:index_col-2]
                label_test=test_whole.iloc[:, index_col-2]
                test = pd.DataFrame(scale(test), index=test.index, columns=test.columns)
                test_whole.iloc[:, 0:index_col-2]=test

                train_labeled=train_labeled_whole.iloc[:, 0:index_col-2]
                label=train_labeled_whole.iloc[:, index_col-2]
                train_labeled = pd.DataFrame(scale(train_labeled), index=train_labeled.index, columns=train_labeled.columns)
                train_labeled_whole.iloc[:, 0:index_col-2]=train_labeled

                train_unlabeled=train_unlabeled_whole.iloc[:, 0:index_col-2]
                label_unlabled=train_unlabeled_whole.iloc[:, index_col-2]
                train_unlabeled = pd.DataFrame(scale(train_unlabeled), index=train_unlabeled.index, columns=train_unlabeled.columns)
                train_unlabeled_whole.iloc[:, 0:index_col-2]=train_unlabeled

                time=train_unlabeled_whole.iloc[:, index_col-1]
                time=time/86400
                confidence_error=np.exp(-1*time/memory_strength)
                train_unlabeled_whole['confidence_error']=confidence_error

            else:
                train_labeled_whole=train_labeled_whole_reference
                train_unlabeled_whole=train_unlabeled_whole_reference
                train_labeled=train_labeled_whole.iloc[:, 0:index_col-2]
                label=train_labeled_whole.iloc[:, index_col-2]
                train_unlabeled=train_unlabeled_whole.iloc[:, 0:index_col-2]
                label_unlabled=train_unlabeled_whole.iloc[:, index_col-2]
                confidence_error=confidence_error_reference

            ############################################
            for xx in range(Budget-B2):
                svm_model_linear = SVC(kernel = 'linear', C = 1, probability=True).fit(train_labeled, label)
                predictions1_prob = svm_model_linear.predict_proba(train_unlabeled)
                predictions1 = svm_model_linear.predict(train_unlabeled)
                entropy=entroy_calculate(predictions1_prob)


                evaluate_factor= entropy* confidence_error
                train_unlabeled_whole['evaluate_factor']=evaluate_factor

                train_unlabeled_whole=train_unlabeled_whole.sort_values(by=['evaluate_factor'], ascending=False)
                train_unlabeled_whole=train_unlabeled_whole.reset_index(drop=True)
                train_unlabeled=train_unlabeled_whole.iloc[:, 0:index_col-2]
                label_unlabled=train_unlabeled_whole.iloc[:, index_col-2]
                confidence_error=train_unlabeled_whole.iloc[:, index_col]

                xx1=xx+B2
                query_label_assign=query_label_assign_calculate(confidence_error,label_unlabled,xx1, myrandomlist)

                train_labeled_whole=train_labeled_whole.append(train_unlabeled_whole.iloc[0, 0:index_col])
                train_labeled_whole=train_labeled_whole.reset_index(drop=True)
                train_labeled_whole.iloc[-1, index_col-2]=query_label_assign
                train_labeled=train_labeled_whole.iloc[:, 0:index_col-2]
                label=train_labeled_whole.iloc[:, index_col-2]

                train_unlabeled_whole=train_unlabeled_whole.drop([0])
                train_unlabeled_whole=train_unlabeled_whole.reset_index(drop=True)

                #############CALCULATE ACCURACY
                #####Finally predict output and measure accuracy (confusion matrix)
            svm_model_linear = SVC(kernel = 'linear', C = 1, probability=True).fit(train_labeled, label)
            my_predictions = svm_model_linear.predict(test.iloc[:, 0:index_col-2])
    #         label_test=test.iloc[:, index_col-2]
            my_accuracy=accuracy_score(my_predictions, label_test)
            my_precision=precision_score(my_predictions, label_test, average='macro')
            my_recall=recall_score(my_predictions, label_test, average='macro')
            my_f1=f1_score(my_predictions, label_test, average='macro')

            acc_result_sum+=my_accuracy
            presic_result_sum+=my_precision
            recall_result_sum+=my_recall
            f1_result_sum+=my_f1

            #####cerate reference sets
            if myx==myiteration-1:
                train_labeled_whole_reference=train_labeled_whole
                train_unlabeled_whole_reference=train_unlabeled_whole
                confidence_error_reference=train_unlabeled_whole_reference.iloc[:, index_col]


        acc_average=acc_result_sum/float(myiteration)
        presic_average=presic_result_sum/float(myiteration)
        recall_average=recall_result_sum/float(myiteration)
        f1_average=f1_result_sum/float(myiteration)

        acc_all.append(acc_average)
        presic_all.append(presic_average)
        recall_all.append(recall_average)
        f1_all.append(f1_average)
        B2=Budget#############################################INNNNNNNNNNNNNNNNNNNNNNNN
    print(acc_all)
    print(presic_all)
    print(recall_all)
    print(f1_all)

##test plot
plt.plot(Budget_range,acc_all, c="blue", label='entropy by error')
plt.show()

In [4]:
def entroy_calculate(predictions1_prob):
    entropy=[None] * len(predictions1_prob)
    for j in range(len(predictions1_prob)):
        entropy[j]=sc.stats.entropy(predictions1_prob[j])
    return entropy

In [5]:
def query_label_assign_calculate(confidence_error,label_unlabled,xx1, myrandomlist):
    query_label_assign=''
    query_real_label=label_unlabled[0]
    
    myrandom=myrandomlist[xx1]
    if myrandom <= confidence_error[0]*100:
        query_label_assign=query_real_label
    else:
        myrandom2=np.random.randint(0,9)
        if query_real_label!=activities[myrandom2]:
            query_label_assign=activities[myrandom2]
        elif myrandom2!=9:
            query_label_assign=activities[myrandom2+1]
        elif myrandom2==9:
            query_label_assign=activities[0]
    return query_label_assign